# Pyramid Poolingモジュールの解説と実装

In [5]:
#パッケージのインポート
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models,transforms
import cv2 
import time

In [3]:
class conv2DBatchNormRelu(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size,stride,padding,dilation,bias):
        super(conv2DBatchNormRelu,self).__init__()
        self.conv=nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding,dilation,bias=bias)
        self.batchnorm=nn.BatchNorm2d(out_channels)
        self.relu=nn.ReLU(inplace=True)
        #inplace設定で、入力を保存せずに出力を計算し、メモリ削減する

    def forward(self,x):
        x=self.conv(x)
        x=self.batchnorm(x)
        outputs=self.relu(x)

        return outputs

## クラスPyramidPoolingの実装

In [6]:
class PyramidPooling(nn.Module):
    def __init__(self,in_channels,pool_sizes,height,width):
        super(PyramidPooling).__init__()

        #forwardで使用する画像サイズ
        self.height=height
        self.width=width

        #各畳み込み層の出力チャネル数
        out_channels=int(in_channels/len(pool_sizes))

        #各畳み込み層を作成
        #pool_sizes:[6,3,2,1]
        self.avpool_1=nn.AdaptiveAvgPool2d(output_size=pool_sizes[0])
        self.cbr_1=conv2DBatchNormRelu(in_channels,out_channels,kernel_size=1,
                                    stride=1,padding=0,dilation=1,bias=False)

        self.avpool_2=nn.AdaptiveAvgPool2d(output_size=pool_sizes[1])
        self.cbr_2=conv2DBatchNormRelu(in_channels,out_channels,kernel_size=1,
                                    stride=1,padding=0,dilation=1,bias=False) 

        self.avpool_3=nn.AdaptiveAvgPool2d(output_size=pool_sizes[2])
        self.cbr_3=conv2DBatchNormRelu(in_channels,out_channels,kernel_size=1,
                                    stride=1,padding=0,dilation=1,bias=False)  

        self.avpool_4=nn.AdaptiveAvgPool2d(output_size=pool_sizes[3])
        self.cbr_4=conv2DBatchNormRelu(in_channels,out_channels,kernel_size=1,
                                    stride=1,padding=0,dilation=1,bias=False)   


    def  forward(self,x):
        
        out1=self.cbr_1(self.avpool_1(x))
        out1=F.interpolate(out1, size=(
            self.height,self.width),mode="bilinear",align_corners=True)

        out2=self.cbr_2(self.avpool_2(x))
        out1=F.interpolate(out2, size=(
            self.height,self.width),mode="bilinear",align_corners=True)

        out3=self.cbr_3(self.avpool_3(x))
        out3=F.interpolate(out3, size=(
            self.height,self.width),mode="bilinear",align_corners=True)

        out4=self.cbr_4(self.avpool_4(x))
        out4=F.interpolate(out1, size=(
            self.height,self.width),mode="bilinear",align_corners=True)

        #最終的に結合させる、dim=1でチャンネル数の次元で結合
        output=torch.cat([x,out1,out2,out3,out4],dim=1)

        return output


        